In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import skew, kurtosis
from statsmodels.tsa import stattools
from sklearn.model_selection import train_test_split

np.set_printoptions(threshold=np.nan)

In [2]:
dataset = np.loadtxt('Dataset_Erick.csv', delimiter=",")
x_train = dataset[:, 1:] # 2D Array
labels = dataset[:, 0]   # 1D Array

In [3]:
CLASSES = 5
y_train = np.zeros([len(labels), CLASSES])

for i in range(len(labels)):
    y_train[i][int(labels[i])] = 1

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.1)

In [4]:
INPUT_LENGHT = x_train.shape[1] # 56 
HIDDEN_UNITS = 1000

Win = np.random.normal(size=[INPUT_LENGHT, HIDDEN_UNITS])

In [5]:
def input_to_hidden(x):
    a = np.dot(x, Win)
    a = np.maximum(a, 0, a) # ReLU
    return a

In [6]:
X = input_to_hidden(x_train)
Xt = np.transpose(X)

Wout = np.dot(np.linalg.pinv(np.dot(Xt, X)), np.dot(Xt, y_train))
# np.linalg.inv(np.dot(Xt, X))
# np.dot(Xt, X)
# Wout

In [7]:
def predict(x):
    x = input_to_hidden(x)
    y = np.dot(x, Wout)
    return y

In [8]:
y = predict(x_test)
correct = 0
total = y.shape[0]

temp = np.zeros([total])

for i in range(total):
    predicted = np.argmax(y[i])
    test = np.argmax(y_test[i])
    temp[i] = np.argmax(y[i])
    correct = correct + (1 if predicted == test else 0)
print('Accuracy: {:f}'.format(correct/total))

Accuracy: 0.967742


In [9]:
temp

array([0., 1., 2., 0., 3., 0., 2., 3., 1., 3., 4., 0., 2., 3., 4., 1., 0.,
       0., 3., 2., 2., 1., 0., 1., 1., 0., 3., 1., 0., 1., 4.])

In [10]:
COLUMNS = ['timestamp', 'xAxis', 'yAxis', 'zAxis']
WALKING_241018_1 = pd.read_csv('Train_1462518004872.csv', header=None, names=COLUMNS)

def magnitude(activity):
    x2 = activity['xAxis'] * activity['xAxis']
    y2 = activity['yAxis'] * activity['yAxis']
    z2 = activity['zAxis'] * activity['zAxis']
    m2 = x2 + y2 + z2
    m = m2.apply(lambda x: math.sqrt(x))
    return m

WALKING_241018_1['magnitude'] = magnitude(WALKING_241018_1)

def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)
    
def plot_activity(activity):
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
    plot_axis(ax0, activity['timestamp'], activity['xAxis'], 'x Axis')
    plot_axis(ax1, activity['timestamp'], activity['yAxis'], 'y Axis')
    plot_axis(ax2, activity['timestamp'], activity['zAxis'], 'z Axis')
    plt.subplots_adjust(hspace=0.2)
    plt.show()
    
def windows(df, size=100):
    start = 0
    while start < df.count():
        yield start, start + size
        start += (size / 2)
        
def plot_magnitudes(activities, titles, window=False):
    fig, axs = plt.subplots(nrows=len(activities), figsize=(15, 15))
    for i in range(0, len(activities)):
        plot_axis(axs[i], activities[i]['timestamp'], activities[i]['magnitude'], titles[i])
        if window == True:
            for (start, end) in windows(activities[i]['timestamp']):
                axs[i].axvline(activities[i]['timestamp'][start], color='r')
    plt.subplots_adjust(hspace=0.2)
    plt.show()
    
# https://en.wikipedia.org/wiki/Jitter
def jitter(axis, start, end):
    j = float(0)
    for i in range(start, min(end, axis.count())):
        if start != 0:
            j += abs(axis[i] - axis[i-1])
    return j / (end-start)

def mean_crossing_rate(axis, start, end):
    cr = 0
    m = axis.mean()
    for i in range(start, min(end, axis.count())):
        if start != 0:
            p = axis[i-1] > m
            c = axis[i] > m
            if p != c:
                cr += 1
    return float(cr) / (end-start-1)

def window_summary(axis, start, end):
    acf = stattools.acf(axis[start:end])   # https://en.wikipedia.org/wiki/Autocorrelation
    acv = stattools.acovf(axis[start:end]) # https://en.wikipedia.org/wiki/Autocovariance
    sqd_error = (axis[start:end] - axis[start:end].mean()) ** 2
    return [
        jitter(axis, start, end),             # Jitter
        mean_crossing_rate(axis, start, end), # Mean Crossing Rate
        axis[start:end].mean(),               # Mean
        axis[start:end].std(),                # Standard Deviation                  # https://en.wikipedia.org/wiki/Standard_deviation
        axis[start:end].var(),                # Variance                            # https://en.wikipedia.org/wiki/Variance
        axis[start:end].min(),                # Minimum
        axis[start:end].max(),                # Maximum
        acf.mean(),                           # mean auto correlation
        acf.std(),                            # standard deviation auto correlation
        acv.mean(),                           # mean auto covariance
        acv.std(),                            # standard deviation auto covariance
        skew(axis[start:end]),                # skewness                            # https://en.wikipedia.org/wiki/Skewness
        kurtosis(axis[start:end]),            # kurtosis                            # https://en.wikipedia.org/wiki/Kurtosis
        math.sqrt(sqd_error.mean())           # Square Root Deviation
    ]

def features(activity):
    for (start, end) in windows(activity['timestamp']):
        features = []
        for axis in ['xAxis', 'yAxis', 'zAxis', 'magnitude']:
            features += window_summary(activity[axis], int(start), int(end))
        yield features
        
erick_test = list(features(WALKING_241018_1))

c = predict(erick_test)
total = c.shape[0]
temp1 = np.zeros([total])
temp2 = np.zeros([total])
# len(y_test)
for i in range(total):
    predicted = np.argmax(c[i])
#     test = np.argmax(y_test[i])
    temp1[i] = np.argmax(c[i])
#     temp2[i] = np.argmax(y_test[i])
#     correct = correct + (1 if predicted == test else 0)
    
# print('Accuracy: {:f}'.format(correct/total))

In [11]:
temp1

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 0., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 0., 0., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4.])